# Peer Graded Assignment Week 3

## Importing all the required Libraries

In [2]:
import numpy as np
import pandas as pd
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import re
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

## Importing the data 

In [37]:
# specify the url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [38]:
# query the website and return the html to the variable ‘page’
page = urlopen(url)

In [39]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page, "html.parser")

In [40]:
#Define array to hold all of the data points
Toronto = []

In [41]:
#Get the first table in the html
data = soup.findAll('table')

In [42]:
#assign the cells to the array
for row in data:
    for item in row.findAll('td'):
        if "<td>" in str(item):
            Toronto.append(str(item))

In [43]:
#Remove the last element in the list as it is invalid
Toronto.pop()   

'<td></td>'

In [44]:
#Clean up the tags and data points
#Remove HTML tags
def remove_tags(Toronto_list):
    tags = ["<td>", "</td>", "\n", "td>" , "</td", "]]"]
    for i in range(0, len(Toronto_list)):
        for j in range(0, len(tags)):
            if str(tags[j]) in str(Toronto_list[i]):
                Toronto_list[i] = Toronto_list[i].replace(tags[j], "")
                if 'title="' in str(Toronto_list[i]):
                    Toronto_list[i] = str(Toronto_list[i]).split('title="')[1].split('">')[0]
    
    return (Toronto_list)
Toronto = remove_tags(Toronto)

In [45]:
#Compile postal codes
def compile_postal(Toronto_list): # This function recursively groups the postal codes and information such as the neighborhoods

    #Compare the postal code to the next one in order
    for i in range (0, len(Toronto_list)-3, 3):

        if str(Toronto_list[i]) == str(Toronto_list[i+3]):
            #Add to the current postal code
            if str(Toronto_list[i+4]) not in Toronto_list[i+1]:
                Toronto_list[i+1] = str(Toronto_list[i+1]) + ", " + str(Toronto_list[i+4])
            if str(Toronto_list[i+5]) not in Toronto_list[i+2]:
               Toronto_list[i+2] = str(Toronto_list[i+2]) + ", " + str(Toronto_list[i+5])
            
            #Remove old entry(s)
            del(Toronto_list[i+3])
            del(Toronto_list[i+3])
            del(Toronto_list[i+3])
            
            Toronto_list = compile_postal(Toronto_list)
            
            break
            
    return Toronto_list

Toronto = compile_postal(Toronto)

In [46]:
#Drop Not assigned boroughs
def drop_na_borough(Toronto_list):

    for i in range (1, len(Toronto_list)-1, 3):
        if str(Toronto_list[i]) == 'Not assigned':
            
            #Remove the row
            del(Toronto_list[i-1])
            del(Toronto_list[i-1])
            del(Toronto_list[i-1])
            
            Toronto_list = drop_na_borough(Toronto_list)
            break
            
    return Toronto_list

Toronto = drop_na_borough(Toronto)

In [47]:
#Assign borough to n/a neighborhoods
def neighborhood_borough(Toronto_list):
    
    for i in range (2, len(dToronto_list), 3):
        if str(Toronto_list[i]) == 'Not assigned':
            
            Toronto_list[i] = str(Toronto_list[i-1])
            Toronto_list = neighborhood_borough(Toronto_list)
            
            break
            
    return Toronto_list

Toronto = neighborhood_borough(Toronto)

NameError: name 'dToronto_list' is not defined

## Transforming into pandas dataframe

In [20]:
#Create a dictionary
toronto_dict = {'Postal_Code':Toronto[0::3], 'Borough': Toronto[1::3], 
                                     'Neighborhood':Toronto[2::3] }

In [21]:
#Pandas Data frame
toronto_df = pd.DataFrame.from_dict(toronto_dict)

In [22]:
print(toronto_df.shape)

(103, 3)


In [23]:
toronto_df.head()

,Borough,Neighborhood,Postal_Code
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront (Toronto), Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Queen's Park (Toronto),Queen's Park (Toronto),M7A


## Using the Geocoder Package

In [29]:
from geopy.geocoders import Nominatim

dfgeo = pd.read_csv("http://cocl.us/Geospatial_data")
dfgeo.rename(columns={'Postal Code': 'Postal_Code'}, inplace=True)

df2 = pd.merge(toronto_df, dfgeo, on="Postal_Code", how='left')

df2.shape

df2.head()

,Borough,Neighborhood,Postal_Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Harbourfront (Toronto), Regent Park",M5A,43.654260,-79.360636
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,Queen's Park (Toronto),Queen's Park (Toronto),M7A,43.662301,-79.389494


## Generating map to visualize neighborhood

In [32]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.653963, -79.387207.


In [35]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto